In [2]:
%load_ext autoreload
%autoreload 2
from utils import*
from preprocessing import preprocessing,preprocessing2
from feature_extraction import extract_features,read_images
from training import *
from performance import performance_analysis
from itertools import compress



In [3]:
features_vector_train=np.empty((0,4608))
Y_train=np.array([])
features_vector_test=np.empty((0,4608))
Y_test=np.array([])


Preprocessing

In [4]:
def preRGB(img):
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    R=img[:,:,0]
    G=img[:,:,1]
    B=img[:,:,2]

    BGR_Max=np.maximum.reduce([R,G,B])
    BGR_Min=np.minimum.reduce([R,G,B])

    Rule_1=np.logical_and.reduce([R>95,G>40,B>20,(BGR_Max-BGR_Min)>15,abs(R-G)>15,R>G,R>B])
    Rule_2=np.logical_and.reduce([R>220,G>210,B>170,abs(R-G)<=15,R>B,G>B])

    RGB_Rule=np.bitwise_or(Rule_1,Rule_2)
    RGB_Rule=RGB_Rule*255


    # Flip
    sum_cols = np.sum(RGB_Rule,axis=0)

    OCR=sum_cols
    res=list(compress(range(len(sum_cols==np.max(OCR))),sum_cols==np.max(OCR)))

    # print(np.sum(RGB_Rule!=0))
    # print(np.sum(RGB_Rule!=255))
    # print(np.shape(RGB_Rule)[0]*np.shape(RGB_Rule)[1])
    # print(np.max(RGB_Rule))


    if(res[0]<(np.shape(img)[1]/2)):
        RGB_Rule=cv2.flip(RGB_Rule,1)

    return RGB_Rule


    # # Find Contours
    # contours, hierarchy = cv2.findContours(
    #     RGB_Rule, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    # # Get Largest Contour
    # sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
    # largest_contour = sorted_contours[0]

    # # Binary_img_contours[Result]
    # hand_contour = np.zeros((np.shape(img)[0], np.shape(img)[1], 1))
    # cv2.drawContours(hand_contour, largest_contour, -1, 255, 10)

    # return hand_contour




def OCR(features_vector,img,name):
    '''img:Binary Assume images are same size :D'''
    sum_cols = np.sum(img,axis=0)
    if(np.shape(sum_cols)[0]!=np.shape(features_vector)[1]):
        print("Mismatch in Size",name," has ",np.shape(sum_cols)[0] ,'while before has ',np.shape(features_vector)[1])
        return features_vector
    
    features_vector=np.vstack([features_vector,sum_cols])
    return features_vector

In [6]:
import matplotlib.cm as cm
import numpy as np

#Training Data Set
for i in range(0, 6):
    print(i)
    path_of_the_directory='../data_split/split/men/train/'+str(i)+'/'
    path_of_the_directory_result='../outputs/preprocessing_output/men/train/'+str(i)+'/'
    isExist = os.path.exists(path_of_the_directory_result)
    if not isExist:
        os.makedirs(path_of_the_directory_result)
        print("The new directory is created!")
    else:
        shutil.rmtree(path_of_the_directory_result, ignore_errors=False, onerror=None)
        os.mkdir(path_of_the_directory_result)
    # ind=0
    for filename in os.listdir(path_of_the_directory):
        # if(ind>20):
        #     break
        # ind=ind+1
        img = cv2.imread(path_of_the_directory+str(filename))
        if img is None:
            continue

        # Skin Masks
        binary_hand=preRGB(img)
        features_vector_train=OCR(features_vector_train,binary_hand,filename)
        Y_train=np.append(Y_train,i)
        # binary_hand=preprocessing(img, gamma=True, close=True)
        # binary_hand=preprocessing2(img)

        # plt.imsave(path_of_the_directory_result+filename, np.array(binary_hand), cmap=cm.gray)

        # cv2.imwrite(path_of_the_directory_result+filename,binary_hand)

    path_of_the_directory='../data_split/split/women/train/'+str(i)+'/'
    path_of_the_directory_result='../outputs/preprocessing_output/women/train/'+str(i)+'/'
    isExist = os.path.exists(path_of_the_directory_result)
    if not isExist:
        os.makedirs(path_of_the_directory_result)
        print("The new directory is created!")
    # ind=0
    for filename in os.listdir(path_of_the_directory):
        # if(ind>20):
        #     break
        # ind=ind+1
        img = cv2.imread(path_of_the_directory+str(filename))
        if img is None:
            continue

        # Skin Masks
        binary_hand=preRGB(img)
        features_vector_train=OCR(features_vector_train,binary_hand,filename)
        Y_train=np.append(Y_train,i)
        # binary_hand=preprocessing(img, gamma=True, close=True)
        # binary_hand=preprocessing2(img)

        # plt.imsave(path_of_the_directory_result+filename, np.array(binary_hand), cmap=cm.gray)

        # cv2.imwrite(path_of_the_directory_result+filename,binary_hand)

0
1
2
3
4
5


In [10]:
# # creating df object with columns specified   
# csv = pd.DataFrame(features_vector_train, columns =list(np.shape(features_vector_train)[1])) 

# # #Adding Lables to the Data Frame
# csv.insert(0, "class", Y_train)
# # print(csv)


# csv.to_csv (r'../outputs/train.csv', index = None)

# print(np.shape(features_vector_train))



np.save('../outputs/train_x',features_vector_train)
np.save('../outputs/train_y',Y_train)


In [12]:
#Testing Data Set
for i in range(0, 6):
    print(i)
    path_of_the_directory='../data_split/split/men/test/'+str(i)+'/'
    path_of_the_directory_result='../outputs/preprocessing_output/men/test/'+str(i)+'/'
    isExist = os.path.exists(path_of_the_directory_result)
    if not isExist:
        os.makedirs(path_of_the_directory_result)
        print("The new directory is created!")
    # ind=0
    for filename in os.listdir(path_of_the_directory):
        # if(ind>20):
        #     break
        # ind=ind+1
        img = cv2.imread(path_of_the_directory+str(filename))
        if img is None:
            continue

        # Skin Masks
        binary_hand=preRGB(img)
        features_vector_test=OCR(features_vector_test,binary_hand,filename)
        Y_test=np.append(Y_test,i)
        # binary_hand=preprocessing(img, gamma=True, close=True)
        # binary_hand=preprocessing2(img)
        # cv2.imwrite(path_of_the_directory_result+filename,binary_hand)

    path_of_the_directory='../data_split/split/women/test/'+str(i)+'/'
    path_of_the_directory_result='../outputs/preprocessing_output/women/test/'+str(i)+'/'
    isExist = os.path.exists(path_of_the_directory_result)
    if not isExist:
        os.makedirs(path_of_the_directory_result)
        print("The new directory is created!")
    # ind=0
    for filename in os.listdir(path_of_the_directory):
        # if(ind>20):
        #     break
        # ind=ind+1
        img = cv2.imread(path_of_the_directory+str( filename))
        if img is None:
            continue

        # Skin Masks
        binary_hand=preRGB(img)
        features_vector_test=OCR(features_vector_test,binary_hand,filename)
        Y_test=np.append(Y_test,i)
        # binary_hand=preprocessing(img, gamma=True, close=True)
        # binary_hand=preprocessing2(img)
        # cv2.imwrite(path_of_the_directory_result+filename,binary_hand)

0
The new directory is created!
1
The new directory is created!
The new directory is created!
2
The new directory is created!
The new directory is created!
3
The new directory is created!
The new directory is created!
4
The new directory is created!
The new directory is created!
5
The new directory is created!
The new directory is created!


In [13]:
# csv = pd.DataFrame(features_vector_test, columns =list(np.shape(features_vector_test)[1])) 

# # #Adding Lables to the Data Frame
# csv.insert(0, "class", Y_train)
# # print(csv)


# csv.to_csv (r'../outputs/train.csv', index = None)


np.save('../outputs/test_x',features_vector_test)
np.save('../outputs/test_y',Y_test)


Feature Extraction

In [ ]:
# Step(1) Read Images
path='../outputs/preprocessing_output/'
train_images=read_images(path,type="train")

print("Read 0:", np.shape(train_images['0']))
print("Read 1:", np.shape(train_images['1']))
print("Read 2:", np.shape(train_images['2']))
print("Read 3:", np.shape(train_images['3']))
print("Read 4:", np.shape(train_images['4']))
print("Read 5:", np.shape(train_images['5']))


test_images=read_images(path,type="test")

print("Read 0:", np.shape(test_images['0']))
print("Read 1:", np.shape(test_images['1']))
print("Read 2:", np.shape(test_images['2']))
print("Read 3:", np.shape(test_images['3']))
print("Read 4:", np.shape(test_images['4']))
print("Read 5:", np.shape(test_images['5']))



In [ ]:
number_of_clusters=8
path='../outputs/preprocessing_output/'
X_train,Y_train,visual_words=extract_features(path,number_of_clusters,debug=True,images=train_images,train=True,visual_words=None)
# print(np.shape(visual_words)



In [ ]:
# creating df object with columns specified   
csv = pd.DataFrame(X_train, columns =list(range(number_of_clusters))) 

# #Adding Lables to the Data Frame
csv.insert(0, "class", Y_train)
# print(csv)


csv.to_csv (r'../outputs/train.csv', index = None)

In [ ]:
path='../outputs/preprocessing_output/'
X_test,Y_test,visual_words=extract_features(path,clusters=None,debug=True,images=test_images,train=False,visual_words=visual_words)

In [ ]:
# creating df object with columns specified   
csv = pd.DataFrame(X_test, columns =list(range(number_of_clusters))) 

#Adding Lables to the Data Frame

csv.insert(0, "class", Y_test)
print(csv)


csv.to_csv (r'../outputs/test.csv', index = None)

Training

In [ ]:
# Read from CVS File
df = pd.read_csv('../outputs/train.csv')
# print(df)


classes_train=df.loc[:,"class"]
# classes_train = classes_train.to_numpy()
# print(classes_train)
# print(np.shape(classes_train))


features_train=df.loc[:, df.columns != 'class']
# features_train = features_train.to_numpy()
# print(features_train)
print(np.shape(features_train))

# Read from CVS File
df = pd.read_csv('../outputs/test.csv')
# print(df)


classes_test=df.loc[:,"class"]
# classes_test = classes_test.to_numpy()
# print(classes_test)
# print(np.shape(classes_test))


features_test=df.loc[:, df.columns != 'class']
# features_test = features_test.to_numpy()
# print(features_test)
print(np.shape(features_test))



### Bag of Visual Words(BoVW)
##### SIFT TO SVM 

In [15]:
# prediction, actual = train_randomforest(features_train,classes_train,features_test,classes_test)
# prediction, actual = train_svm(features_train,classes_train,features_test,classes_test)

prediction, actual = train_randomforest(features_vector_train,Y_train,features_vector_test,Y_test)
accuracy = performance_analysis(prediction,actual)
print(accuracy)
prediction, actual = train_svm(features_vector_train,Y_train,features_vector_test,Y_test)
accuracy = performance_analysis(prediction,actual)
print(accuracy)

# prediction, actual = train_svm(features_train,classes_train,features_test,classes_test)



# print(prediction)
# print(actual)

0.6717948717948717
0.40512820512820513


Accuracy

In [ ]:
accuracy = performance_analysis(prediction,actual)
print(accuracy)